<a href="https://colab.research.google.com/github/ynkim0/Stock_and_Financial_Statements/blob/main/Financial_statement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
from io import BytesIO
import zipfile
import xmltodict

api = 'https://opendart.fss.or.kr/api/corpCode.xml'
res = requests.get(api, params={'crtfc_key': '9a360d3e3365eba4c46575c83fe03b7947088252'})
data_xml = zipfile.ZipFile(BytesIO(res.content))

https://proceedings.neurips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf

In [ ]:
data_xml.namelist()

['CORPCODE.xml']

In [ ]:
import xmltodict
import json

data_xml = data_xml.read('CORPCODE.xml').decode('utf-8')
data_odict = xmltodict.parse(data_xml)
data_dict = json.loads(json.dumps(data_odict))
data = data_dict.get('result', {}).get('list')

In [ ]:
for item in data:
    if item['corp_name'] in ["삼성전자", "SK하이닉스", "NAVER"]:
        print(item)

{'corp_code': '00126380', 'corp_name': '삼성전자', 'stock_code': '005930', 'modify_date': '20220509'}
{'corp_code': '00164779', 'corp_name': 'SK하이닉스', 'stock_code': '000660', 'modify_date': '20220331'}
{'corp_code': '00266961', 'corp_name': 'NAVER', 'stock_code': '035420', 'modify_date': '20220728'}


http://blog.quantylab.com/2020-10-09-dart.html

In [ ]:
crtfc_key = '9a360d3e3365eba4c46575c83fe03b7947088252'
 
import requests 
import pandas as pd 
import io 
import zipfile
import xml.etree.ElementTree as et 
import json

In [ ]:
def get_corpcode(crtfc_key): 
    """ 
    OpenDART 기업 고유번호 받아오기 
    return 값: 주식코드를 가진 업체의 DataFrame 
    """ 
    params = {'crtfc_key':crtfc_key} 
    items = ["corp_code","corp_name","stock_code","modify_date"] 
    item_names = ["고유번호","회사명","종목코드","수정일"] 
    url = "https://opendart.fss.or.kr/api/corpCode.xml" #요청 url
    res = requests.get(url,params=params) #url 불러오기
    zfile = zipfile.ZipFile(io.BytesIO(res.content))  #zip file 받기
    fin = zfile.open(zfile.namelist()[0])  #zip file 열고
    root = et.fromstring(fin.read().decode('utf-8'))  #utf-8 디코딩
    data = [] 
    for child in root: 
        if len(child.find('stock_code').text.strip()) > 1: # 종목코드가 있는 경우 
            data.append([]) #data에 append하라 
            for item in items: 
                data[-1].append(child.find(item).text) 
    df = pd.DataFrame(data, columns=item_names) 
    return df

In [ ]:
stock_comp = get_corpcode(crtfc_key)
stock_comp

,고유번호,회사명,종목코드,수정일
0,00260985,한빛네트,036720,20170630
1,00264529,엔플렉스,040130,20170630
2,00358545,동서정보기술,055000,20170630
3,00231567,애드모바일,032600,20170630
4,00247939,씨모스,037600,20170630
...,...,...,...,...
3515,00185505,제일바이오,052670,20221122
3516,00993931,동양파일,228340,20221124
3517,00115287,동방,004140,20221124
3518,00557933,진도,088790,20221108


In [ ]:
code_list = stock_comp['고유번호'].tolist()

In [ ]:
!pip install dart_fss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 126 kB 25.4 MB/s 
     |████████████████████████████████| 41 kB 486 kB/s 
     |████████████████████████████████| 50 kB 6.1 MB/s 
     |████████████████████████████████| 1.9 MB 43.2 MB/s 
     |████████████████████████████████| 61 kB 89 kB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
  Created wheel for arelle: filename=arelle-2.2-py3-none-any.whl size=2060307 sha256=f996f30d21302cc77a8f4cd781f6d9839e60754c0abfd1a0d4dbeb76f41df124
  Stored in directory: /root/.cache/pip/wheels/36/af/2e/a5a7c700dcb6ca300f86d2bc6674c6d37704baacf1deac116d
  Created wheel for halo: filename=halo-0.0.31-py3-none-any.whl size=11261 sha256=cf79a95689dd1c19ae2486a6ddd50374ec726d87c07d1e2bb6f4a8cc7dbb88c1
  Stored in directory: /root/.cache/pip/wheels/c0/74/6f/c63bb0075c49f05e692807fe383ddf2b17a52f567d637af096
Successfully built arelle halo
  Attempting uninsta

In [ ]:
import dart_fss as dart
 
 
# Open DART API KEY 설정
api_key = '9a360d3e3365eba4c46575c83fe03b7947088252'
dart.set_api_key(api_key=api_key)

'9a360d3e3365eba4c46575c83fe03b7947088252'

In [ ]:
def get_fs(crtfc_key, corp_code, bsns_year, reprt_code):
    # 단일회사 주요계정
    # crtfc_key API 인증키 STRING(40)  Y 발급받은 인증키(40자리)
    # corp_code 고유번호  STRING(8) Y 공시대상회사의 고유번호(8자리)
    # ※ 개발가이드 > 공시정보 > 고유번호 API조회 가능
    # bsns_year 사업연도  STRING(1) Y 사업연도(4자리)
    # ※ 2015년 이후 부터 정보제공
    # reprt_code  보고서 코드  STRING(5) Y 1분기보고서 : 11013
    # 반기보고서 : 11012
    # 3분기보고서 : 11014
    # 사업보고서 : 11011
    api = 'https://opendart.fss.or.kr/api/fnlttSinglAcnt.json'
    params = {
        'crtfc_key': crtfc_key,
        'corp_code': corp_code,
        'bsns_year': bsns_year,
    }
    data = []
    reprt_name = REPRT_NAMES.get(reprt_code, "")
    params['reprt_code'] = reprt_code
    res = self._request(api, params=params)
    if res is None:
        return None
    if res.status_code != 200:
        return None
    item = json.loads(res.text)
    if item.get('status', '') != '000':
        return None
    for _item in item.get('list'):
        _item['corp_code'] = corp_code
        _item['reprt_code'] = reprt_code
        _item['reprt_name'] = reprt_name
        data.append(_item)
    return data


In [ ]:
#삼성 재무정보파일 호출
file='fs_samsung.xlsx'
wb=load_workbook(file)
 
#필요한 데이터 레이블
bs_items=['ifrs-full_Assets','ifrs-full_Liabilities','ifrs-full_Equity']
is_items=['ifrs-full_Revenue','dart_OperatingIncomeLoss','ifrs-full_ProfitLoss']
cf_items=['ifrs-full_CashFlowsFromUsedInOperatingActivities']
 
ws_bs=wb['Data_bs']
y_list=[]
 
 
c= 10
 
while ws_bs.cell(2,c).value != None:
    y=ws_bs.cell(1,c).value
    y_list.append(y)
    c+=1
print(y_list)
 
 
df_list=[]
 
for bs_item in bs_items:
    temp_list=[]
    r=4
    while ws_bs.cell(r,2).value !=None:
        if ws_bs.cell(r,2).value==bs_item:
            item=ws_bs.cell(r,3).value
            c=10
            while ws_bs.cell(1,c).value !=None:
                temp_list.append(ws_bs.cell(r,c).value)
                c+=1
        r+=1
    df=pd.DataFrame({item:temp_list},index=y_list)
    df_list.append(df)
 
ws_is=wb['Data_is']
for is_item in is_items:
    temp_list=[]
    r=4
    while ws_is.cell(r,2).value !=None:
        if ws_is.cell(r,2).value==is_item:
            item=ws_is.cell(r,3).value
            c=8
            while ws_is.cell(1,c).value !=None:
                temp_list.append(ws_is.cell(r,c).value)
                c+=1
        r+=1
    df=pd.DataFrame({item:temp_list},index=y_list)
    df_list.append(df)
 
ws_cf=wb['Data_cf']
for cf_item in cf_items:
    temp_list=[]
    r=4
    while ws_cf.cell(r,2).value !=None:
        if ws_cf.cell(r,2).value==cf_item:
            item=ws_cf.cell(r,3).value
            c=9
            while ws_cf.cell(1,c).value !=None:
                temp_list.append(ws_cf.cell(r,c).value)
                c+=1
        r+=1
    df=pd.DataFrame({item:temp_list},index=y_list)
    df_list.append(df)
 
total_df=pd.concat(df_list,axis=1)
total_df=total_df.transpose()
 
# col 생략 없이 출력
pd.set_option('display.max_columns', None)
print(total_df)
 
#파일 저장
#total_df.to_excel('재무정보상태표.xlsx')

https://hanfit.tistory.com/7
https://gils-lab.tistory.com/44
https://besixdouze.net/13
https://weezzle.tistory.com/1379